In [ ]:
#Install Anthropic
%pip install anthropic

In [ ]:
#Import libraries
import anthropic
import csv
import json
import uuid
from datetime import datetime
from io import StringIO

# Initialize Anthropic client
client = anthropic.Anthropic(api_key="API_KEY")

# Patient data CSV
PATIENT_DATA_CSV = """Id,Name,Surname,Credit Card,Address,Tier,Country
1,Alice,Johnson,"4000 1234 5678 9010","14 Baker St, Apt 2B",Advanced,USA
2,Bob,Smith,"4111 2345 6789 0121","789 Pine Ln",Simple,Canada
3,Charlie,Brown,"4222 3456 7890 1232","50 Oak Ave",Premium,UK
4,Diana,Miller,"4333 4567 8901 2343","101 Elm Blvd",Advanced,Australia
5,Ethan,Wilson,"4444 5678 9012 3454","22 Maple Rd",Simple,Germany
6,Fiona,Clark,"4555 6789 0123 4565","33 Cedar Pkwy",Premium,France
7,George,Harris,"4666 7890 1234 5676","44 Birch Dr",Simple,USA
8,Hannah,Lewis,"4777 8901 2345 6787","55 Spruce Cir",Advanced,Canada
9,Ian,Walker,"4888 9012 3456 7898","66 Poplar Ct",Simple,UK
10,Jasmine,Hall,"4999 0123 4567 8909","77 Willow Wy",Premium,Australia"""

# Medical Policy Document
MEDICAL_POLICY = """
Comprehensive Insurance Policy Document: 2024 Edition
Section 1: Membership Tiers Overview

1. Simple Membership
- Monthly Fee: $29.99
- Core Coverage: Basic medical and emergency services only.
- Deductible: $1,000 per incident.
- Dental/Vision: Not included.
- Maximum Annual Payout: $50,000.
- Key Exclusions: Pre-existing conditions, preventative care, and specialized surgery.

2. Advanced Membership
- Monthly Fee: $79.99
- Core Coverage: Comprehensive medical, preventative care (annual physicals), and limited prescription drug coverage.
- Deductible: $500 per incident.
- Dental/Vision: Includes annual dental checkup and $150 towards vision care.
- Maximum Annual Payout: $250,000.
- Key Features: Access to telemedicine services 24/7.

3. Premium Membership
- Monthly Fee: $149.99
- Core Coverage: Full comprehensive medical, unlimited prescription drug coverage, and specialized surgery coverage.
- Deductible: $0 (No deductible).
- Dental/Vision: Full coverage for all routine and major dental/vision services.
- Maximum Annual Payout: Unlimited.
- Key Features: Global emergency travel assistance and a dedicated customer service agent.
"""

In [ ]:
#Define action logging
def log_action(action, case_id, tool_used={}, comment={}):
    """Log actions with timestamp and case ID"""
    log_entry = {
        "action": action,
        "timestamp": datetime.now().isoformat(),
        "caseId": case_id,
        "tool_used": tool_used,
        "comment": comment
    }
    logs.append(log_entry)
    print(f"[LOG] {json.dumps(log_entry)}")

#Define patient data loading
def load_patient_data():
    """Load patient data from CSV string"""
    patients = []
    csv_file = StringIO(PATIENT_DATA_CSV)
    reader = csv.DictReader(csv_file)
    for row in reader:
        patients.append(row)
    return patients

#Define patient search
def find_patient(identifier, patients):
    """Find patient by ID, name, or surname"""
    identifier = str(identifier).strip().lower()

    for patient in patients:
        if (str(patient['Id']) == identifier or
            patient['Name'].lower() == identifier or
            patient['Surname'].lower() == identifier or
            f"{patient['Name'].lower()} {patient['Surname'].lower()}" == identifier):
            return patient
    return None

#Define agent tools
def create_agent_tools():
    """Define tools for the AI agent"""
    tools = [
        {
            "name": "verify_patient",
            "description": "Verify patient identity by checking their ID, name, or surname against the patient database. Returns patient information if found.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "identifier": {
                        "type": "string",
                        "description": "Patient ID, name, or surname to verify"
                    }
                },
                "required": ["identifier"]
            }
        },
        {
            "name": "check_coverage",
            "description": "Check if a patient's policy tier covers their medical circumstance based on the policy document.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "The verified patient ID"
                    },
                    "tier": {
                        "type": "string",
                        "description": "The patient's policy tier (Simple, Advanced, or Premium)"
                    },
                    "circumstance": {
                        "type": "string",
                        "description": "The medical circumstance or service the patient is asking about"
                    }
                },
                "required": ["patient_id", "tier", "circumstance"]
            }
        },
        {
            "name": "finalize_decision",
            "description": "Finalize the coverage decision after verification and evaluation. Must be called to close the case.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "The patient's ID"
                    },
                    "decision": {
                        "type": "string",
                        "enum": ["Covered", "Not Covered", "Escalated"],
                        "description": "The final coverage decision"
                    },
                    "reasoning": {
                        "type": "string",
                        "description": "Brief explanation of the decision"
                    }
                },
                "required": ["patient_id", "decision", "reasoning"]
            }
        }
    ]

    mining_activity_name = {
        tools[0].get("name"): "Patient Verified",
        tools[1].get("name"): "Coverage Checked",
        tools[2].get("name"): "Decision Finalized",
    }
    return tools, mining_activity_name

def process_tool_call(tool_name, tool_input, tool_mining_activity_name, patients, case_id):

    if tool_name == "verify_patient":
        identifier = tool_input['identifier']
        patient = find_patient(identifier, patients)

        if patient:
            log_action(f"{tool_mining_activity_name}", case_id, f"Tool used: {tool_name}")
            return {
                "found": True,
                "patient_id": patient['Id'],
                "name": f"{patient['Name']} {patient['Surname']}",
                "tier": patient['Tier'],
                "country": patient['Country']
            }
        else:
            log_action("Patient not found", case_id, f"Tool used: {tool_name}")
            return {
                "found": False,
                "message": "Patient not found in database"
            }

    elif tool_name == "check_coverage":
        patient_id = tool_input['patient_id']
        tier = tool_input['tier']
        circumstance = tool_input['circumstance'].lower()

        log_action(f"{tool_mining_activity_name}", case_id, f"Tool used: {tool_name}")

        #log_action(f"Checking coverage for: {circumstance}", case_id)

        # Coverage logic based on policy document
        coverage_rules = {
            "Simple": {
                "covered": ["basic medical", "emergency services", "emergency"],
                "not_covered": ["pre-existing", "preventative", "specialized surgery", "dental", "vision", "prescription"]
            },
            "Advanced": {
                "covered": ["medical", "preventative", "annual physical", "prescription", "dental checkup", "vision", "telemedicine"],
                "not_covered": ["specialized surgery", "major dental"]
            },
            "Premium": {
                "covered": ["medical", "prescription", "specialized surgery", "dental", "vision", "preventative", "emergency", "travel"],
                "not_covered": []
            }
        }

        rules = coverage_rules.get(tier, {})

        # Check if covered
        is_covered = None
        for keyword in rules.get("covered", []):
            if keyword in circumstance:
                is_covered = True
                break

        # Check if explicitly not covered
        for keyword in rules.get("not_covered", []):
            if keyword in circumstance:
                is_covered = False
                break

        if is_covered is None:
            return {
                "result": "unclear",
                "message": "Not enough information to determine coverage"
            }
        elif is_covered:
            return {
                "result": "covered",
                "tier": tier,
                "message": f"{tier} tier covers this service"
            }
        else:
            return {
                "result": "not_covered",
                "tier": tier,
                "message": f"{tier} tier does not cover this service"
            }

    elif tool_name == "finalize_decision":
        patient_id = tool_input['patient_id']
        decision = tool_input['decision']
        reasoning = tool_input['reasoning']

        log_action(f"{tool_mining_activity_name}", case_id, f"Tool used: {tool_name}", f"decision: {decision}")
        log_action("Case closed", case_id)

        return {
            "status": "closed",
            "decision": decision,
            "reasoning": reasoning
        }

    return {"error": "Unknown tool"}

def run_coverage_agent(user_message):
    """Run the agentic AI coverage verification system"""
    # Generate unique case ID for this workflow execution
    case_id = str(uuid.uuid4())

    log_action("Coverage inquiry initiated", case_id)
    log_action("Input message set", case_id)

    patients = load_patient_data()
    tools, mining_activity_name = create_agent_tools()

    system_prompt = f"""You are an insurance coverage verification agent. Your job is to:
1. Ask patients to identify themselves if they haven't provided their ID, name, or surname
2. Verify patient identity using the verify_patient tool
3. If patient not found, ask them to re-enter their information
4. Once verified, evaluate their coverage question based on their policy tier
5. Use the check_coverage tool to determine if they're covered
6. Provide a final decision: "Covered", "Not Covered", or "Escalated" (if unclear)
7. Always finalize with the finalize_decision tool to close the case

Medical Policy Information:
{MEDICAL_POLICY}

Be helpful, professional, and concise. Always use tools to verify information and make decisions."""

    messages = [{"role": "user", "content": user_message}]

    print(f"\n{'='*60}")
    print(f"CASE ID: {case_id}")
    print(f"USER: {user_message}")
    print(f"{'='*60}\n")

    while True:
        response = client.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=4096,
            tools=tools,
            system=system_prompt,
            messages=messages
        )

        # Process response
        assistant_message = {"role": "assistant", "content": response.content}
        messages.append(assistant_message)

        # Check if there's text content to display
        for block in response.content:
            if block.type == "text":
                print(f"AGENT: {block.text}\n")

        # Check if we should stop
        if response.stop_reason == "end_turn":
            break

        # Process tool uses
        if response.stop_reason == "tool_use":
            tool_results = []

            for block in response.content:
                if block.type == "tool_use":
                    tool_name = block.name
                    tool_input = block.input
                    tool_mining_activity_name = mining_activity_name.get(block.name)

                    print(f"[TOOL CALL] {tool_name}: {json.dumps(tool_input, indent=2)}\n")

                    # Execute tool
                    result = process_tool_call(tool_name, tool_input, tool_mining_activity_name, patients, case_id)

                    print(f"[TOOL RESULT] {json.dumps(result, indent=2)}\n")

                    tool_results.append({
                        "type": "tool_result",
                        "tool_use_id": block.id,
                        "content": json.dumps(result)
                    })

            # Add tool results to messages
            messages.append({"role": "user", "content": tool_results})
        else:
            break

    print(f"\n{'='*60}")
    print(f"CONVERSATION ENDED - CASE ID: {case_id}")
    print(f"{'='*60}\n")

    return logs, case_id


In [ ]:
# Log storage
logs = []

# Create another variable that relates uuid and entity id

# Example usage
if __name__ == "__main__":
    print("Insurance Coverage AI Agent System")
    print("="*60)

    all_logs = []

    # Example 1: Patient asks about coverage with identification
    # print("\n### EXAMPLE 1: Patient with ID ###")
    # logs1, case_id1 = run_coverage_agent(
    #     "Hi, I'm patient ID 3. I need to know if my policy covers specialized surgery?"
    # )
    # all_logs.extend(logs1)

    # print("\n### EXAMPLE 2: Patient without identification ###")
    # logs2, case_id2 = run_coverage_agent(
    #     "I need to know if my insurance covers dental work."
    # )
    # all_logs.extend(logs2)

    # print("\n### EXAMPLE 3: Patient with name ###")
    # logs3, case_id3 = run_coverage_agent(
    #     "This is Alice Johnson. Am I covered for preventative care and annual physicals?"
    # )
    # all_logs.extend(logs3)

    print("\n### EXAMPLE 4: Patient with name ###")
    logs4, case_id4 = run_coverage_agent(
        "I'm patient ID 3 and my name is Antonio Perico. Am I covered for preventative care and annual physicals?"
    )
    all_logs.extend(logs4)

    # Print summary of all logs
    print("\n" + "="*60)
    print("COMPLETE LOG SUMMARY")
    print("="*60)
    print(f"\nTotal workflow executions: 3")
    print(f"Case IDs: {case_id4}\n")

    for log in all_logs:
        print(json.dumps(log, indent=2))

In [ ]:
print(json.dumps(logs3, indent=2))